# **Importing libraries**

In [1]:
import os
import pandas as pd
from datetime import datetime
import requests
import json
from db import Session, Base, engine
from schemas import vra

def date_parser(data_str):
    """
        Format any date in YYYY-MM-DD HH:MM:SS
    Args:
        data_str (str): string with the date

    Returns:
        str: string with the date formatted in YYYY-MM-DD HH:MM:SS
    """
    try:
        # Tenta analisar a data de entrada em diferentes formatos
        data_str = str(data_str)
        if data_str[:2] == '00':
            data_str = '20'+str(data_str)[2:]
        # data = datetime(year=int('20'+data_str[2:4]), month=int(data_str[5:7], 10), day=int(data_str[8:10]), hour=int(data_str[11:13]), minute=int(data_str[14:16]), second=int(data_str[17:19]),microsecond=int(0))
        data = datetime.strptime(data_str, '%Y-%m-%d %H:%M:%S')
        if data_str[:2] == '00':
            print(data_str)
            print(data)
    except ValueError:
        try:
            data = datetime.strptime(data_str, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            try:
                data = datetime.strptime(data_str, '%Y-%m-%d')
            except ValueError:
                try:
                    data = datetime.strptime(data_str, '%d/%m/%Y %H:%M:%S')
                except ValueError:
                    try:
                        data = datetime.strptime(data_str, '%d/%m/%Y %H:%M')
                    except ValueError:
                        try:
                            data = datetime.strptime(data_str, '%d/%m/%Y')
                        except ValueError:
                            try:
                                data = datetime.strptime(data_str, '%Y-%m-%d %H:%M')
                            except ValueError:
                                # print("Data inválida:", data_str)
                                return None  # Retornar None se a data não pôde ser analisada

    # Formate a data no formato desejado
    data_formatada = data.strftime('%Y-%m-%d %H:%M:%S.%f')
    data_obj = datetime.strptime(data_formatada, '%Y-%m-%d %H:%M:%S.%f')
    return data_obj

def trf_dtypes(schema,dataframe):
    """
    Transforma os tipos de dados do dataframe de acordo com o schema especificado.

    Args:
        schema_bq (dict): Dicionário contendo o schema do BigQuery, com os nomes das colunas e os tipos de dados.
        dataframe (pandas.DataFrame): O dataframe a ser transformado.

    Returns:
        dict: Um dicionário contendo as informações do schema e o dataframe transformado.
    """
    # print(dataframe.dtypes)
    for key, value in schema.items():
        # print(f'Parsing {key} for type {value}')
        # dataframe = dataframe.rep
        match(value):
            case 'Integer':
                dataframe[key] = dataframe[key].astype(pd.Int64Dtype())
            case 'DateTime':
                dataframe[key] = dataframe[key].apply(date_parser)
                # dataframe[key] = pd.to_datetime(dataframe[key],format='%Y-%m-%d %H:%M:%S')
            # case 'Float':
            #     dataframe[key] = dataframe[key].apply(trf_numeric)
            #     dataframe[key] = dataframe[key].round(decimals=7).astype(float)
            #     context = decimal.Context(prec=10,Emax=99999, Emin=-99999)
            #     dataframe[key] = dataframe[key].apply(context.create_decimal_from_float)
            case 'String':
                # dataframe[key] = dataframe[key].apply(trf_str)
                dataframe[key] = dataframe[key].replace("null", None)
                dataframe[key] = dataframe[key].astype(pd.StringDtype())
    return dataframe


# **Creation of the SQLite Database**

In [2]:
# # Criação do banco de dados
# session = Session()
# Base.metadata.create_all(engine)


# **Solution for the first question**

In [3]:
print("Starting the process of loading the JSON data")

# Declare the path to the folder containing the JSON files
path = r'source\VRA'

# Declare the variable to store all JSON data
data_json = pd.DataFrame()

# Loop through each JSON file in the folder
for file in os.listdir(path):
    with open(os.path.join(path, file), 'r',encoding='utf-8-sig') as f:
        data = json.load(f)
    print(f'Loaded {file} -> {len(data)} rows')
    data_json = pd.concat([data_json, pd.DataFrame(data)], ignore_index=True)
print(f"Finished loading the JSON data -> {len(data_json)} rows")

# Normalize the headers for snake case pattern
rename_dict = {
    "ICAOEmpresaAérea":"icao_empresa_aerea",
    "NúmeroVoo":"numero_voo",
    "CódigoAutorização":"codigo_autorizacao",
    "CódigoTipoLinha":"codigo_tipo_linha",
    "ICAOAeródromoOrigem":"icao_aerodromo_origem",
    "ICAOAeródromoDestino":"icao_aerodromo_destino",
    "PartidaPrevista":"partida_prevista",
    "PartidaReal":"partida_real",
    "ChegadaPrevista":"chegada_prevista",
    "ChegadaReal":"chegada_real",
    "SituaçãoVoo":"situacao_voo",
    "CódigoJustificativa":"codigo_justificativa"}

data_json.rename(columns=rename_dict,inplace=True)
# print(data_json)

schema = {
    # 'id':'Integer',
    'icao_empresa_aerea':'String',
    'numero_voo':'String',
    'codigo_autorizacao':'String',
    'codigo_tipo_linha':'String',
    'icao_aerodromo_origem':'String',
    'icao_aerodromo_destino':'String',
    'partida_prevista':'DateTime',
    'partida_real':'DateTime',
    'chegada_prevista':'DateTime',
    'chegada_real':'DateTime',
    'situacao_voo':'String',
    'codigo_justificativa':'String'
}

# Transforming datetime data types
# data_json = trf_data(['partida_prevista','partida_real','chegada_prevista','chegada_real'],data_json)
print("Transforming the data types")
print('Before:\n',data_json.dtypes)
data_json = trf_dtypes(schema,data_json)
print('After:\n',data_json.dtypes)
# data_json = data_json.fillna(None)
data_json = data_json.replace(pd.NA,None)
data_json = data_json.where(pd.notna(data_json), None)
print("Saving to the database")
# dado = vra(**data_json) 
# dado = vra(**data_json.to_dict(orient='records')) 
# session.add(dado)

# Itere pelas linhas do DataFrame
# for index, row in data_json.iterrows():
#     # Crie uma instância da classe vra com base nos valores da linha
    
#     dado = vra(**row.to_dict())

#     # Adicione a instância à sessão
#     session.add(dado)

# session.commit()
# Saving the data as a SQLite file
# data_json.to_sql('vra', con='sqlite:///vra.db', index=False,if_exists='replace')
data_json.to_sql('vra', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace')



<>:75: SyntaxWarning: invalid escape sequence '\d'
<>:75: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gusta\AppData\Local\Temp\ipykernel_23436\1384376854.py:75: SyntaxWarning: invalid escape sequence '\d'
  data_json.to_sql('vra', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace')


Starting the process of loading the JSON data
Loaded VRA_20211.json -> 58550 rows
Loaded VRA_202110.json -> 59430 rows
Loaded VRA_202111.json -> 64004 rows
Loaded VRA_20212.json -> 43692 rows
Loaded VRA_20213.json -> 39514 rows
Loaded VRA_20214.json -> 29122 rows
Loaded VRA_20215.json -> 35750 rows
Loaded VRA_20216.json -> 40955 rows
Loaded VRA_20217.json -> 53313 rows
Loaded VRA_20218.json -> 54928 rows
Loaded VRA_20219.json -> 56545 rows
Finished loading the JSON data -> 535803 rows
icao_empresa_aerea        object
numero_voo                object
codigo_autorizacao        object
codigo_tipo_linha         object
icao_aerodromo_origem     object
icao_aerodromo_destino    object
partida_prevista          object
partida_real              object
chegada_prevista          object
chegada_real              object
situacao_voo              object
codigo_justificativa      object
dtype: object
Parsing icao_empresa_aerea for type String
Parsing numero_voo for type String
Parsing codigo_autoriz

535803

In [4]:
# from sqlalchemy import create_engine, Column, Integer, DateTime
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import sessionmaker
# from datetime import datetime
# from sqlalchemy import inspect

# # Defina a classe do modelo com uma coluna de data e hora
# Base = declarative_base()

# class Exemplo(Base):
#     __tablename__ = 'exemplo'
#     id = Column(Integer, primary_key=True)
#     data_hora = Column(DateTime)

# # Crie uma conexão com o banco de dados SQLite
# engine = create_engine('sqlite:///exemplo.db')

# # Crie as tabelas no banco de dados
# Base.metadata.create_all(engine)

# # Crie uma instância da classe do modelo e atribua None à coluna de data e hora
# exemplo = Exemplo(data_hora=None)

# # Inicie uma sessão do SQLAlchemy
# Session = sessionmaker(bind=engine)
# session = Session()

# # Adicione o objeto à sessão e cometa a transação
# session.add(exemplo)
# session.commit()

# # Verifique o resultado
# inspector = inspect(engine)
# columns = inspector.get_columns('exemplo')
# for column in columns:
#     print(f"{column['name']}: {getattr(exemplo, column['name'])}")
